<a href="https://colab.research.google.com/github/BreakoutMentors/Data-Science-and-Machine-Learning/blob/main/machine_learning/lesson%204%20-%20ML%20Apps/Gradio/Pretrained_Model_Gradio_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Always open in Colab for the best learning experience.

# Image Classification App using Gradio

In the last lesson, we used Transfer Learning to use a pretrained [ResNet101](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet101) model on our own dataset. In this lesson here, we will use that model that was saved and downloaded to our computers to create an ML app that classifies images of flowers for us using Gradio.

This challenge is inspired by Gradio's example called [*Image Classification with PyTorch*](https://colab.research.google.com/drive/1S6seNoJuU7_-hBX5KbXQV4Fb_bbqdPBk?usp=sharing), so please feel free to refer to that as an extra resource.

These are the steps necessary to build the Gradio app:
1. Upload the model's parameters
2. Loading the model
3. Getting labels of dataset
4. Defining the Input into our model
5. Defining the Output of our model
6. Defining the function that uses your model
7. Compiling the Interface

## Importing all the libraries needed

In [ ]:
!pip install -q gradio
import gradio as gr
import requests

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np

## Upload your model's parameters

Google colab will provide you a prompt below to upload the model from the previous lesson.

In [ ]:
from google.colab import files
files.upload()

Saving saved_model.pt to saved_model.pt


## Loading the Model

Before loading the model's parameters saved in the file `saved_model.pt`, you have to load the exact pretrained model that you used before. After loading the model, you have to alter the parameters just like how it was done before when you finetuned it, so you can see that the `model.fc` layer was adjusted just as before. To load in the parameters saved from the file, you use `torch.load` function and map it to the 'cpu' device to use the model on the colab's cpu instead of gpu since we are not training the model. After loading the models parameters, we can now replace the current paramaters with the saved paramters using the `load_state_dict` method.



In [ ]:
model = torchvision.models.resnet101(pretrained=True)
saved_model_parameters = torch.load("saved_model.pt", map_location=torch.device('cpu'))

num_features = model.fc.in_features
num_classes = 5

# Replacing model.fc layer
model.fc = torch.nn.Sequential(
           nn.Linear(num_features, 512),
           nn.Linear(512, num_classes)
)

model.load_state_dict(saved_model_parameters)

# Placing model in evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Getting Labels of dataset

Below is a list of the classes in order of how the dataset was trained. This list is going to be used to match your labels to the probabilities of the output of your model that are displayed in the app with `gradio.outputs.Label` output.

In [ ]:
labels = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

## Defining the Input into our model

We will be using the `gradio.inputs.Image` class that allows the user of your application to load in images to classify. The list below discusses the parameters used.

The parameters of the input:
1. **Shape (tuple)** - (width, height) shape to crop and resize image to; if None, matches input image size.
2. **image_mode (str)** - "RGB" if color, or "L" if black and white.
3. **source (str)** - Source of image. "upload" creates a box where user can drop an image file, "webcam" allows user to take snapshot from their webcam, "canvas" defaults to a white image that can be edited and drawn upon with tools.
4. **type (str)** - Type of value to be returned by component. "numpy" returns a numpy array with shape (width, height, 3) and values from 0 to 255, "pil" returns a PIL image object, "file" returns a temporary file object whose path can be retrieved by file_obj.name.


We are also going to be using image examples! Gradio allows us to use images on your machine to have users choose those images to classify rather than uploading their own, which is pretty cool! Gradio does this by using the `examples` parameter with the `gradio.Interface` class.

In [ ]:
# Defining the input
input_img = gr.inputs.Image(shape=(224, 224),
                            image_mode='RGB',
                            source='upload',
                            type='pil')

In [ ]:
# Getting examples
!wget https://github.com/BreakoutMentors/Data-Science-and-Machine-Learning/raw/main/machine_learning/lesson%204%20-%20ML%20Apps/images/Transfer_Learning_Images.zip
!unzip Transfer_Learning_Images

sample_images = [['rose_1.jpg'],
                 ['Sunflower_1.jpeg'],
                 ['tulip_1.jpeg'],
                 ['dandelion_1.jpeg'],
                 ['daisy_1.jpeg']]

--2021-06-21 06:56:27--  https://github.com/BreakoutMentors/Data-Science-and-Machine-Learning/raw/adam-transfer-learning/machine_learning/lesson%204%20-%20ML%20Apps/images/Transfer_Learning_Images.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BreakoutMentors/Data-Science-and-Machine-Learning/adam-transfer-learning/machine_learning/lesson%204%20-%20ML%20Apps/images/Transfer_Learning_Images.zip [following]
--2021-06-21 06:56:27--  https://raw.githubusercontent.com/BreakoutMentors/Data-Science-and-Machine-Learning/adam-transfer-learning/machine_learning/lesson%204%20-%20ML%20Apps/images/Transfer_Learning_Images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

## Defining the Output of our model

Since we are classifying images, the output of the model we have are probabilities of our classes. Gradio has an output class called `gradio.outputs.Label` that shows the label with the highest probability. The output class has a parameter called `num_top_classes` that gives the developer the choice to choose how many classes with the highest probabilities for the interface to display. Since we only have 5 classes for this dataset, we will just choose 5 classes.

In [ ]:
output = gr.outputs.Label(num_top_classes=5)

## Defining the function that uses your model

The function below called `classify_image` will take the input which is a [`PIL Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html) and we use transforms that were used on the training images that were responsible of converting the images into tensors and then using the same means and standard deviations to normalize them. Again, the means and standard deviations are from the [ImageNet](https://www.image-net.org/) dataset since the pretrained model was trained with that dataset.

The function returns a Python dictionary that contains the name of the classes as the keys and their matching probabilities as the values. This dictonary is what is used for the `gradio.outputs.Label` class to display the classes with their probabilities.

In [ ]:
# Defining transforms to normalize the inputted PIL image to a tensor
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

def classify_image(img):
    # Changes shape from (3, 224, 224) to (1, 3, 224, 224) for the model
    img = transform(img).unsqueeze(0)

    # Calculate output of the model and not calculating gradients
    with torch.no_grad():
        output = model(img)
    
    # Calculating probabilities from the output(logits)
    probabilities = torch.nn.functional.softmax(output, dim=1).flatten()

    # Returning a dictionary with the labels(keys) and their predictions(values)
    return {labels[i]:probabilities[i].item() for i in range(len(labels))}

## Compiling the Interface

Here we are compiling the input, output, function and examples to run our ML application. To launch the interface, use the `launch()` method.

In [ ]:
iface = gr.Interface(fn=classify_image,
                     inputs=input_img,
                     outputs=output,
                     examples=sample_images)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://49746.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://49746.gradio.app')